# Keytanglement Jupyter Notebook

Quantum key distribution (QKD) has the ability to replace traditional methods of encrypted communication and protect against further development in quantum decryption algorithms such as Shor's Algorithm. Keytanglement is a webserver implementation of a Bell state QKD in response to iQuHACK 2022's QuTech challenge.

### The Challenge
The situation of the QuTech QKD Challenge is described by the image below where Alice and Bob are communicating classically and want to generate a key to keep their communications secret.


![AliceBobEve](qkd-alice-bob-eve.png)



 Eve is an eavesdropper and tried to decode the key they are using by snooping on their communications to a quantum computer. Thus, the communications required to get a key must not include enough information for Eve to determine the key. In this scenario, Eve is snooping on the quantum circuit and can insert measurements. A proposed system architecture is included below, we show our system architecture in the Implementation section.

 ![QutechInterface](qutech_sysarch.png)

### The Research
 We implemented a Bell state QKD solution which requires only 11 bits to determine if an eavesdropper exists as opposed to the commonly used BB84 algorithm which requires 72 bits. The paper we used by Song and Chen can be found [here](doi: 10.1109/LCOMM.2020.2988380).

 Their research used a physical quantum system which differs from our software simulation of the QKD problem since Alice and Bob share a physical quantum connection. The original implementation is summarized and the changes are described below.

 Bell states are a superposition of qubits that causes entaglements. The four two-qubit gates are shown below.
 ![BellStates](bell_states.png)

 The original implementation had 4 qubits. 
 1. Alice chooses to randomly entangle two pairs of qubits according to the table below, where G is a group code associated with the superposition. 
 2. Then, she sends the qubits to Bob in a random order.
 3. Bob randomly pairs the qubits and performs a measurement in the Bell basis.
 4. Like BB84, he sends Alice information on his measurements, and she checks for correctness.
 5. They share some pairing information to ensure no eavesdropper performed a measurement between Alice and Bob.
 6. They extract the key from their pairing information and communicate!

 ![Gtable](group_codes.png)


 ### Our implementation

The main issue of implementing the QKD above is that we do not understand how to perform steps 2 and 3 using Qiskit. Instead we derived a different but similar method to extract information from the Bell states.

 1. Alice chooses to randomly entangle two pairs of qubits according to the table below, where G is a group code associated with the superposition according to the table below.
 2. **Bob randomly pairs the qubits, and performs a reverse operation of a group code.**
 3. **Due to quantum circuit symmetry properties, if Bob performs the exact reverse of Alice's circuit, 
    the computational basis measurement will be the initial state or |0000>.**
 4. Like BB84, he sends Alice information on his measurements that are correct (have a state vector of |0000>).
 5. They share some pairing information to ensure no eavesdropper performed a measurement between Alice and Bob.
 6. They extract the key from the **group codes of the correct guesses** and communicate!

### Our final product

We created a web server that allows users to generate QKDs then use them to talk over encrypted chat. The repository is located [here](https://github.com/JRice15/keytanglement). The users go through the steps detailed in the image below [INSERT] to get their key and start encrypted communications.


### Test Cases
We implemented testing along the way but to demonstrate the workings of our QDK method here are some testcases for different pairings and groupings. A pairing refers to how the qubits are assigned to each entangled bell state, and the states are chosen via the group codes in the table above.

The first test case is where Alice and Bob choose the same pairing and the resulting state vector is all 0s. 
The second test case is where Alice and Bob choose different pairings but the same grouping.
The third test case is where Alice and Bob choose the same pairings but not the same grouping.

In [4]:
from qiskit import QuantumCircuit, Aer, assemble
from math import pi
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
# In Jupyter Notebooks we can display this nicely using Latex.
# If not using Jupyter Notebooks you may need to remove the 
# array_to_latex function and use print() instead.
from qiskit_textbook.tools import array_to_latex
from jupyter_notebook import display

In [ ]:
#Identical pairing + GC Case 
# Alice's and Bob's chocies for qubit-pairs and group codes
alice_pairs = [[0, 1], [2, 3]]
alice_groupcode = 3 # 11
bob_pairs = [[0, 1], [2, 3]]
bob_groupcode = 3 # 11

qc = QuantumCircuit(4)

# Alice's circuit

# Psi-minus
qc.h(0)
qc.x(1)
qc.z(0)
qc.z(1)
qc.cx(0,1)

# Psi-plus
qc.h(2)
qc.x(3)
qc.cx(2,3)

# Bob's reversal circuit


# Psi-minus reversal
qc.cx(0,1)
qc.z(1)
qc.z(0)
qc.x(1)
qc.h(0)

# Psi-plus reversal
qc.cx(2,3)
qc.x(3)
qc.h(2)

display(qc.draw())

# Let's see the result
svsim = Aer.get_backend('statevector_simulator')
qobj = assemble(qc)
final_state = svsim.run(qobj).result().get_statevector()
array_to_latex(final_state, pretext="\text{Statevector} = ", precision=1)

NameError: name 'QuantumCircuit' is not defined

In [ ]:

#Qubit Pairs Differ Case 
# Alice's and Bob's chocies for qubit-pairs and group codes
alice_pairs = [[0, 2], [1, 3]]
alice_groupcode = 3 # 11
bob_pairs = [[0, 1], [2, 3]]
bob_groupcode = 3 # 11

qc = QuantumCircuit(4)

# Alice's circuit

# Psi-minus
qc.h(0)
qc.x(2)
qc.z(0)
qc.z(2)
qc.cx(0,2)

# Psi-plus
qc.h(1)
qc.x(3)
qc.cx(1,3)

# Bob's reversal circuit

# Psi-minus reversal
qc.cx(0,1)
qc.z(1)
qc.z(0)
qc.x(1)
qc.h(0)

# Psi-plus reversal
qc.cx(2,3)
qc.x(3)
qc.h(2)

display(qc.draw())

# Let's see the result
svsim = Aer.get_backend('statevector_simulator')
qobj = assemble(qc)
final_state = svsim.run(qobj).result().get_statevector()
array_to_latex(final_state, pretext="\text{Statevector} = ", precision=1)

In [ ]:
#Differing GCs Case
# Alice's and Bob's chocies for qubit-pairs and group codes
alice_pairs = [[0, 1], [2, 3]]
alice_groupcode = 1 # 01
bob_pairs = [[0, 1], [2, 3]]
bob_groupcode = 3 # 11

qc = QuantumCircuit(4)

# Alice's circuit

# Phi-minus
qc.x(0)
qc.h(0)
qc.cx(0,1)

# Phi-plus
qc.h(2)
qc.cx(2,3)

# Bob's reversal circuit

# Psi-minus reversal
qc.cx(0,1)
qc.z(1)
qc.z(0)
qc.x(1)
qc.h(0)

# Psi-plus reversal
qc.cx(2,3)
qc.x(3)
qc.h(2)

display(qc.draw())

# Let's see the result
svsim = Aer.get_backend('statevector_simulator')
qobj = assemble(qc)
final_state = svsim.run(qobj).result().get_statevector()
array_to_latex(final_state, pretext="\text{Statevector} = ", precision=1)